# LlamaIndex와 AgentCore Memory - Investment Portfolio Advisor (장기 메모리)

## 소개

이 노트북은 Amazon Bedrock AgentCore Memory 기능을 LlamaIndex와 통합하여 **장기 메모리** 지속성을 갖춘 Investment Portfolio Advisor를 생성하는 방법을 보여줍니다. 여러 고객 미팅과 시장 사이클에 걸쳐 누적된 투자 지식을 구축하고 수개월 및 수년에 걸친 포트폴리오 성과를 추적할 수 있습니다.

## 아키텍처 개요

![LlamaIndex AgentCore Long-Term Memory Architecture](LlamaIndex-AgentCore-LTM-Arch.png)

## 튜토리얼 세부사항

**튜토리얼 세부사항:**
- **튜토리얼 유형**: 장기 크로스 세션 메모리
- **Agent 사용 사례**: Investment Portfolio Advisor
- **Agentic Framework**: LlamaIndex
- **LLM model**: Anthropic Claude 3.7 Sonnet
- **튜토리얼 구성 요소**: AgentCore 장기 메모리, LlamaIndex Agent, Financial Tools
- **예제 복잡도**: 고급

## 비즈니스 가치

**엔터프라이즈 투자 인텔리전스**: 포트폴리오 지식을 축적하고, 투자 진화를 추적하며, 분기와 연도에 걸쳐 포괄적인 시장 분석을 유지하는 지속적인 AI 메모리로 자산 관리 업무를 혁신하세요.

**주요 전문적 이점:**
- **포트폴리오 연속성**: 투자 기간과 팀 구성원 간의 원활한 지식 전달
- **투자 메모리**: 중요한 시장 인사이트, 전략 및 성과 데이터를 영구적으로 보존
- **크로스 포트폴리오 인텔리전스**: 여러 고객 포트폴리오에 걸친 패턴과 연결 식별
- **전략적 우수성**: 우수한 투자 결정을 위한 과거 성과 데이터 활용
- **고객 관계**: 다년간의 자산 관리를 위한 상세한 컨텍스트 유지
- **리스크 관리**: 시장 사이클과 투자 전략에 대한 영향 추적

## 장기 메모리 구성

**기술 설정**: 이 튜토리얼은 12개월 보존 기간을 가진 Semantic Strategy와 함께 AgentCore Memory를 사용합니다:
- **메모리 유형**: 자동 인사이트 추출을 위한 Semantic strategy
- **보존 기간**: 포트폴리오 연속성을 위한 365일 이벤트 만료
- **크로스 세션**: 동일한 actor_id + memory_id, 투자 기간별로 다른 session_id
- **검색 기능**: 포트폴리오 히스토리 전반에 걸친 시맨틱 검색을 위한 내장 메모리 검색 도구

## 기술 개요

**주요 장기 메모리 구성 요소:**
1. **Semantic Strategy 구성**: 365일 보존 기간을 가진 자동 인사이트 추출을 위해 SemanticStrategy 사용
2. **크로스 세션 지속성**: 동일한 actor_id + memory_id, 기간별로 다른 session_id로 지식 연속성 활성화
3. **커스텀 메모리 검색 도구**: AgentCore의 네이티브 search_long_term_memories()를 LlamaIndex FunctionTool로 래핑
4. **시맨틱 처리 파이프라인**: 대화 이벤트 → 시맨틱 메모리 변환을 위한 90초 대기
5. **동적 세션 관리**: 유연한 세션 처리를 위해 memory.context.session_id 사용

**학습 내용:**

- 여러 고객 미팅에 걸쳐 지속적인 AgentCore Memory 생성
- 시간 경과에 따른 누적 투자 지식 구축
- 시장 조사 및 고객 히스토리 전반에 걸친 시맨틱 검색 구현
- 포트폴리오 진화 및 투자 성과 추적
- 크로스 세션 금융 지식 지속성 및 검색 테스트

## 시나리오 컨텍스트

이 예제에서는 분기와 연도에 걸친 여러 미팅에서 고객 투자 지식을 유지하는 "Investment Portfolio Advisor"를 생성합니다. Advisor는 AgentCore Memory를 사용하여 시간이 지남에 따라 성장하고 진화하는 고객 프로필, 포트폴리오 성과, 시장 인사이트 및 투자 결과의 지속적인 지식 베이스를 구축하여 정교한 종단 자산 관리를 가능하게 합니다.

## 사전 요구 사항

- Python 3.10+
- 적절한 권한을 가진 AWS 계정
- AgentCore Memory 권한을 가진 AWS IAM 역할:
  - `bedrock-agentcore:CreateMemory`
  - `bedrock-agentcore:CreateEvent`
  - `bedrock-agentcore:ListEvents`
  - `bedrock-agentcore:RetrieveMemories`
- Amazon Bedrock models에 대한 액세스

## 단계 1: 종속성 설치 및 설정

In [ ]:
# 필요한 라이브러리 설치 (semantic strategy toolkit 포함)
%pip install llama-index-memory-bedrock-agentcore llama-index-llms-bedrock-converse boto3 bedrock-agentcore-starter-toolkit

In [ ]:
# 필요한 컴포넌트 import
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory.session import MemorySessionManager
from bedrock_agentcore_starter_toolkit.operations.memory.models.strategies.semantic import SemanticStrategy
from llama_index.memory.bedrock_agentcore import AgentCoreMemory, AgentCoreMemoryContext
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import FunctionTool
from datetime import datetime
import os

print("✅ All dependencies imported successfully!")

## 단계 2: AgentCore Memory 구성

장기 투자 지식을 위한 AgentCore Memory 리소스를 생성하거나 가져옵니다:

In [ ]:
# Semantic Strategy를 사용하여 AgentCore Memory 생성
region = os.getenv('AWS_REGION', 'us-east-1')
memory_manager = MemoryManager(region_name=region)

try:
    # 자동 인사이트 추출을 위한 semantic strategy로 메모리 생성
    memory = memory_manager.get_or_create_memory(
        name=f'InvestmentAdvisorSemantic_{int(datetime.now().timestamp())}',
        strategies=[SemanticStrategy(name="investmentLongTermMemory")],
        event_expiry_days=365  # 투자 기록을 위한 12개월 보존
    )
    memory_id = memory.get('id')
    print(f"✅ Created Semantic Memory: {memory_id}")
    print(f"   Status: {memory.get('status')}")
    print(f"   Strategies: {[s.get('name') if isinstance(s, dict) else str(s) for s in memory.get('strategies', [])]}")
    
    # 메모리가 ACTIVE 상태가 될 때까지 대기
    if memory.get('status') != 'ACTIVE':
        print(f"\n⏳ Waiting for memory to become ACTIVE (currently {memory.get('status')})...")
        import time
        max_wait = 300  # 최대 5분
        waited = 0
        while waited < max_wait:
            time.sleep(10)
            waited += 10
            # 상태 확인
            current_memory = memory_manager.get_memory(memory_id)
            status = current_memory.get('status')
            print(f"   [{waited}s] Status: {status}")
            if status == 'ACTIVE':
                print(f"✅ Memory is now ACTIVE! (took {waited} seconds)")
                break
        else:
            print(f"⚠️  Memory still not ACTIVE after {max_wait}s. Proceeding anyway...")
    
except Exception as e:
    print(f"❌ Error creating memory: {e}")
    memory_id = "your-memory-id-here"  # 기존 memory ID로 교체

## 단계 3: Investment Tools 구현

종단 자산 관리를 위한 전문 도구를 정의합니다:

In [ ]:
def record_client_meeting(client_id: str, meeting_type: str, portfolio_value: str, key_decisions: str) -> str:
    """포트폴리오 업데이트 및 결정사항과 함께 고객 미팅 기록"""
    return f"📅 Recorded {meeting_type} for {client_id} (${portfolio_value})"

def track_portfolio_performance(client_id: str, period: str, return_pct: str, benchmark_return: str, attribution: str) -> str:
    """벤치마크 대비 포트폴리오 성과를 기여도 분석과 함께 추적"""
    return f"📈 {client_id} {period}: {return_pct} vs {benchmark_return}"

def document_market_insight(insight_type: str, market_event: str, impact_assessment: str, client_implications: str) -> str:
    """고객 포트폴리오 영향과 함께 시장 인사이트 문서화"""
    print(f"🌍 Market insight: {insight_type} - {market_event} (Impact: {impact_assessment})")
    return f"Documented market insight: {insight_type}"

def update_investment_thesis(client_id: str, asset_class: str, thesis: str, conviction_level: str) -> str:
    """특정 자산 클래스에 대한 투자 논제 업데이트"""
    print(f"💭 Investment thesis: {client_id} - {asset_class} ({conviction_level} conviction)")
    return f"Updated thesis for {client_id}"

def log_rebalancing_action(client_id: str, action_type: str, securities: str, rationale: str) -> str:
    """근거와 함께 포트폴리오 리밸런싱 액션 기록"""
    print(f"⚖️ Rebalancing: {client_id} - {action_type}: {securities}")
    return f"Logged rebalancing for {client_id}"

def log_advisory_milestone(quarter: str, milestone: str, details: str) -> str:
    """분기 및 상세 진행 상황과 함께 자문 마일스톤 기록"""
    print(f"🎯 {quarter} milestone: {milestone}")
    return f"Logged milestone for {quarter}: {milestone} - {details}"

def track_investment_metrics(metric_type: str, value: str, client_id: str, quarter: str) -> str:
    """고객 및 타임라인과 함께 특정 투자 지표 추적"""
    print(f"📊 {quarter}: {metric_type} = {value} (for {client_id})")
    return f"Tracked {metric_type}: {value} for {client_id} in {quarter}"

def save_advisory_insight(insight: str, quarter: str, market_context: str) -> str:
    """시장 컨텍스트와 함께 자문 인사이트 저장"""
    print(f"💡 {quarter} insight: {insight[:50]}...")
    return f"Saved {quarter} insight with market context: {market_context}"

# Agent를 위한 tool 객체 생성
investment_tools = [
    FunctionTool.from_defaults(fn=record_client_meeting),
    FunctionTool.from_defaults(fn=track_portfolio_performance),
    FunctionTool.from_defaults(fn=document_market_insight),
    FunctionTool.from_defaults(fn=update_investment_thesis),
    FunctionTool.from_defaults(fn=log_rebalancing_action),
    FunctionTool.from_defaults(fn=log_advisory_milestone),
    FunctionTool.from_defaults(fn=track_investment_metrics),
    FunctionTool.from_defaults(fn=save_advisory_insight)
]

print("✅ Investment tools created!")

## 단계 3b: 메모리 검색 도구 추가

Agent가 장기 메모리를 검색할 수 있도록 하는 도구를 생성합니다:

In [ ]:
def create_memory_retrieval_tool(memory_id: str, actor_id: str, region: str):
    """Agent가 자신의 장기 메모리를 검색할 수 있도록 하는 도구 생성"""
    
    def search_long_term_memory(query: str) -> str:
        """고객, 포트폴리오, 과거 결정, 시장 인사이트에 대한 관련 정보를 장기 메모리에서 검색.
        
        다음을 기억해야 할 때 이 도구를 사용:
        - 고객 정보 (포트폴리오 가치, 리스크 프로필, 투자 목표)
        - 과거 투자 결정 및 결과
        - 포트폴리오 성과 히스토리
        - 시장 인사이트 및 적용 사례
        - 투자 논제 및 진화
        
        Args:
            query: 필요한 정보를 설명하는 검색 쿼리 (예: 'CLIENT-001 portfolio', 'investment theses', 'Q1 performance')
        
        Returns:
            장기 메모리에서 가져온 관련 정보
        """
        try:
            from bedrock_agentcore.memory.session import MemorySessionManager
            
            # session manager 생성 (memory_id와 region만 필요)
            session_manager = MemorySessionManager(
                memory_id=memory_id,
                region_name=region
            )
            
            # semantic strategy namespace에서 장기 메모리 검색
            results = session_manager.search_long_term_memories(
                query=query,
                namespace_prefix="/strategies/",  # semantic strategy namespace에서 검색
                top_k=5,
                max_results=10
            )
            
            if not results:
                return "No relevant information found in long-term memory. This might be new information or the memory extraction may still be processing."
            
            # Agent를 위한 결과 포맷팅
            output = "📚 Retrieved from long-term memory:\\n\\n"
            for i, result in enumerate(results, 1):
                # MemoryRecord 객체 - content 속성 접근
                content = getattr(result, 'content', str(result))
                # 매우 긴 콘텐츠 잘라내기
                if len(content) > 300:
                    content = content[:300] + "..."
                output += f"{i}. {content}\\n\\n"
            
            return output
            
        except Exception as e:
            return f"⚠️ Error searching memory: {str(e)}. Proceeding without historical context."
    
    return FunctionTool.from_defaults(fn=search_long_term_memory)

# 메모리 검색 도구 생성
memory_search_tool = create_memory_retrieval_tool(memory_id, "financial-advisor", region)

# tools 리스트에 메모리 검색 추가
investment_tools_with_memory = investment_tools + [memory_search_tool]

print(f"✅ Memory retrieval tool created! Total tools: {len(investment_tools_with_memory)}")
print("   Using namespace: /strategies/ (for semantic strategy compatibility)")


## 단계 3c: 메모리 구성 확인

Semantic strategy가 올바르게 구성되었는지 확인합니다:

In [ ]:
# 메모리 구성 확인
memory_info = memory_manager.get_memory(memory_id)
print(f"Strategies: {memory_info.get('strategies')}")
print(f"Status: {memory_info.get('status')}")
print(f"Name: {memory_info.get('name')}")

# Strategy 세부 정보 표시
strategies = memory_info.get('strategies', [])
for strategy in strategies:
    print(f"\nStrategy Details:")
    print(f"  Name: {strategy.get('name')}")
    print(f"  Type: {strategy.get('type')}")
    print(f"  Status: {strategy.get('status')}")
    print(f"  ID: {strategy.get('strategyId')}")


## 단계 4: 멀티 세션 Agent 구현

다양한 자문 기간을 시뮬레이션하기 위한 헬퍼 함수를 생성합니다:

In [ ]:
# 장기 메모리 구성 (크로스 세션)
MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
ADVISOR_ID = "financial-advisor"  # 모든 세션에서 동일한 advisor

def create_advisory_session(session_name: str):
    """장기 메모리 지속성을 가진 새로운 자문 세션 생성"""
    context = AgentCoreMemoryContext(
        actor_id=ADVISOR_ID,         # 동일한 advisor
        memory_id=memory_id,         # 동일한 메모리 저장소 (장기 메모리 활성화)
        session_id=f"advisory-{session_name}", # 기간별로 다른 세션
        namespace="/wealth-management/"
    )
    
    memory = AgentCoreMemory(context=context)
    llm = BedrockConverse(model=MODEL_ID)
    agent = FunctionAgent(
        tools=investment_tools_with_memory,  # 메모리 검색 기능이 있는 도구 사용
        llm=llm, 
        verbose=True,  # 메모리 검색 시점을 보기 위해 verbose 활성화
        system_prompt="""You are a senior investment advisor with access to long-term memory.
        
CRITICAL: When asked about clients, portfolios, past decisions, or historical information, 
you MUST use the search_long_term_memory tool FIRST before responding.

For example:
- "What clients am I managing?" → Use search_long_term_memory("clients portfolio")
- "What was CLIENT-001's performance?" → Use search_long_term_memory("CLIENT-001 performance")
- "What investment theses do I have?" → Use search_long_term_memory("investment thesis")

Always provide conclusive, complete responses without asking follow-up questions.\n
Execute all requested actions immediately and completely. Provide detailed, professional responses."""
    )
    
    return agent, memory

print("✅ Multi-session Investment Portfolio Advisor setup complete!")


## 단계 5: Q1 자문 세션 - 초기 고객 온보딩

첫 번째 자문 세션을 시작하고 고객 기준선을 설정합니다:

In [ ]:
# === Q1 자문 세션 ===
print("🗓️ === Q1: INITIAL CLIENT ONBOARDING ===")

agent_q1, memory_q1 = create_advisory_session("q1")

# 초기 고객 미팅 기록
response = await agent_q1.run(
    "I'm Senior Advisor Jennifer Walsh. Record client meeting for 'CLIENT-001' with meeting type 'Initial Portfolio Review', "
    "portfolio value '$3,200,000', key decisions 'established moderate-aggressive risk profile, 20-year investment horizon, "
    "target allocation 70% equity/25% fixed income/5% alternatives'.",
    memory=memory_q1
)

print("🎯 Q1 Initial Meeting:")
print(response)

In [ ]:
# 초기 투자 논제 문서화
response = await agent_q1.run(
    "Update investment thesis for 'CLIENT-001': asset class 'US Large Cap Equity', "
    "thesis 'overweight growth stocks due to technological innovation and earnings momentum', conviction level 'high'.",
    memory=memory_q1
)
print("💭 Q1 Equity Thesis:", response)

response = await agent_q1.run(
    "Update investment thesis for 'CLIENT-001': asset class 'Fixed Income', "
    "thesis 'short duration bias due to rising rate environment, focus on credit quality', conviction level 'medium'.",
    memory=memory_q1
)
print("💭 Q1 Bond Thesis:", response)

In [ ]:
# 초기 성과 기준선 추적
response = await agent_q1.run(
    "Track portfolio performance for 'CLIENT-001': period 'Q1 2024', return_pct '+8.2%', "
    "benchmark_return '+7.1%', attribution 'tech overweight +0.8%, duration underweight +0.3%'.",
    memory=memory_q1
)
print("📈 Q1 Performance:", response)

# 이벤트가 저장되었는지 확인
print("\n🔍 Verifying Q1 events were stored...")
try:
    client = MemoryClient(region_name=region)
    events = client.list_events(
        memory_id=memory_id,
        actor_id=ADVISOR_ID,
        session_id=memory_q1.context.session_id
    )
    print(f"✅ Stored {len(events)} conversational events in Q1 session")
except Exception as e:
    print(f"⚠️  Could not verify events: {e}")

# semantic 메모리 처리를 위한 대기 시간
import asyncio
print("\n⏳ Waiting for semantic memory extraction and indexing...")
print("   (AgentCore processes conversational events in the background)")
await asyncio.sleep(90)  # 메모리 추출을 위한 대기 시간 증가 (이전 10초)
print("✅ Memory processing complete - memories should now be searchable")

## 단계 6: Q2 자문 세션 - 시장 변동성 대응

장기 메모리 검색을 테스트하고 시장 변화에 적응합니다:

In [ ]:
# === Q2 자문 세션 ===
print("\n🗓️ === Q2: MARKET VOLATILITY RESPONSE (NEW SESSION) ===")

agent_q2, memory_q2 = create_advisory_session("q2")

# 크로스 세션 고객 회상 테스트 - agent는 search_long_term_memory 도구를 사용해야 함
print("\n🧠 Testing memory retrieval across sessions...")
print("   (Watch for the agent to use search_long_term_memory tool)\n")

response = await agent_q2.run(
    "What clients am I managing? What are their portfolio values, risk profiles, and investment theses?",
    memory=memory_q2
)

print("\n🧠 Q2 Client Recall:")
print(response)
print("\n✅ Expected: CLIENT-001, $3.2M portfolio, moderate-aggressive, growth equity thesis")

In [ ]:
# 시장 변동성 인사이트 문서화
response = await agent_q2.run(
    "Document market insight: insight type 'Geopolitical Risk', market event 'Trade tensions escalation', "
    "impact assessment 'increased volatility, sector rotation from growth to value', "
    "client implications 'review tech overweight, consider defensive positioning'.",
    memory=memory_q2
)
print("🌍 Q2 Market Insight:", response)

# 리밸런싱 대응 기록
response = await agent_q2.run(
    "Log rebalancing action for 'CLIENT-001': action type 'Tactical Adjustment', "
    "securities 'reduced QQQ by 3%, increased VTV (value ETF) by 2%, added VGSH (short treasury) by 1%', "
    "rationale 'defensive positioning due to geopolitical uncertainty, maintain long-term allocation targets'.",
    memory=memory_q2
)
print("⚖️ Q2 Rebalancing:", response)

In [ ]:
# Q2 성과 영향 추적
response = await agent_q2.run(
    "Track portfolio performance for 'CLIENT-001': period 'Q2 2024', return_pct '-2.1%', "
    "benchmark_return '-3.8%', attribution 'defensive positioning +1.2%, value tilt +0.5%'.",
    memory=memory_q2
)
print("📈 Q2 Performance:", response)

# 성과 비교 테스트
response = await agent_q2.run(
    "How did CLIENT-001's Q2 performance compare to Q1? What was the cumulative return and attribution?",
    memory=memory_q2
)
print("📊 Q2 Performance Analysis:")
print(response)
print("\n✅ Expected: Q1 +8.2%, Q2 -2.1%, cumulative ~+5.9%, defensive positioning helped")

## 단계 7: Q3 자문 세션 - 회복 및 투자 논제 업데이트

시장 회복으로 진행하고 투자 접근 방식을 업데이트합니다:

In [ ]:
# === Q3 자문 세션 ===
print("\n🗓️ === Q3: MARKET RECOVERY AND THESIS UPDATE ===")

agent_q3, memory_q3 = create_advisory_session("q3")

# 분기별 검토 미팅 기록
response = await agent_q3.run(
    "Record client meeting for 'CLIENT-001' with meeting type 'Quarterly Review', "
    "portfolio value '$3,450,000', key decisions 'market recovery positioning, increase growth allocation, "
    "add international exposure for diversification'.",
    memory=memory_q3
)
print("📅 Q3 Quarterly Review:", response)

# 시장 진화에 기반한 투자 논제 업데이트
response = await agent_q3.run(
    "Update investment thesis for 'CLIENT-001': asset class 'International Equity', "
    "thesis 'add developed market exposure via VTIAX, emerging markets recovery potential', conviction level 'medium'.",
    memory=memory_q3
)
print("💭 Q3 International Thesis:", response)

In [ ]:
# 포괄적인 투자 히스토리 회상 테스트
response = await agent_q3.run(
    "What is the complete investment history for CLIENT-001? Include all meetings, performance periods, "
    "rebalancing actions, and evolution of investment theses.",
    memory=memory_q3
)
print("📋 Q3 Complete History:")
print(response)
print("\n✅ Expected: Q1 onboarding → Q2 defensive moves → Q3 recovery positioning, all performance data")

## 단계 8: Q4 자문 세션 - 연말 검토 및 계획

시맨틱 검색 및 연간 성과 분석을 테스트합니다:

In [ ]:
# === Q4 자문 세션 ===
print("\n🗓️ === Q4: YEAR-END REVIEW AND PLANNING ===")

agent_q4, memory_q4 = create_advisory_session("q4")

# 연간 성과 추적
response = await agent_q4.run(
    "Track portfolio performance for 'CLIENT-001': period '2024 Annual', return_pct '+12.8%', "
    "benchmark_return '+11.2%', attribution 'tactical positioning +1.1%, sector allocation +0.5%'.",
    memory=memory_q4
)
print("📈 Q4 Annual Performance:", response)

# 시장 인사이트 상관관계 테스트
response = await agent_q4.run(
    "What market insights have I documented this year? How did they impact CLIENT-001's portfolio decisions?",
    memory=memory_q4
)
print("🌍 Q4 Market Insight Analysis:")
print(response)
print("\n✅ Expected: Geopolitical risk insight → defensive positioning → outperformance during volatility")

In [ ]:
# 유사한 포트폴리오 액션에 대한 시맨틱 검색 테스트
response = await agent_q4.run(
    "What rebalancing actions have I taken for CLIENT-001? Which were most effective based on subsequent performance?",
    memory=memory_q4
)
print("⚖️ Q4 Rebalancing Analysis:")
print(response)
print("\n✅ Expected: Q2 defensive moves (QQQ reduction, VTV/VGSH adds) helped during volatility")

## 단계 9: 2년차 Q1 세션 - 다년간 관점

장기 투자 지식 및 고객 관계 진화를 테스트합니다:

In [ ]:
# === 2년차 Q1 자문 세션 ===
print("\n🗓️ === YEAR 2 Q1: MULTI-YEAR PERSPECTIVE ===")

agent_y2q1, memory_y2q1 = create_advisory_session("year2-q1")

# 다년간 포트폴리오 분석
response = await agent_y2q1.run(
    "Analyze CLIENT-001's investment journey: How has their portfolio evolved over the past year? "
    "What were the key decisions and their outcomes?",
    memory=memory_y2q1
)
print("📊 Year 2 Q1 Journey Analysis:")
print(response)
print("\n✅ Expected: $3.2M → $3.45M growth, defensive positioning success, thesis evolution")

In [ ]:
# 투자 논제 진화 추적 테스트
response = await agent_y2q1.run(
    "How have my investment theses for CLIENT-001 evolved? What asset classes have I added and why?",
    memory=memory_y2q1
)
print("💭 Year 2 Q1 Thesis Evolution:")
print(response)
print("\n✅ Expected: Started with US equity/fixed income → added international exposure → evolved based on market conditions")

## 단계 10: 최종 자산 관리 포트폴리오 평가

장기 투자 자문 기능에 대한 포괄적인 테스트:

In [ ]:
# 최종 포괄적인 자산 관리 포트폴리오 쿼리
response = await agent_y2q1.run(
    "Provide my complete wealth management portfolio: all clients with their investment journeys, "
    "performance attribution, market insights applied, rebalancing effectiveness, and thesis evolution. "
    "Include lessons learned and best practices developed.",
    memory=memory_y2q1
)
print("💼 Complete Wealth Management Portfolio:")
print(response)
print("\n✅ Expected: Full CLIENT-001 journey with performance attribution, market timing, and investment evolution")

## 🧪 자동화된 테스트 검증
메모리 통합이 올바르게 작동하는지 검증하기 위해 이 셀들을 실행하세요:

In [ ]:
# 검증 함수를 인라인으로 정의
class TestValidator:
    def __init__(self):
        self.results = {}
    
    def validate_memory_recall(self, response):
        """Agent가 세션 초반의 정보를 회상할 수 있는지 확인"""
        # 실질적인 응답 확인 (단순히 "I don't know"가 아닌)
        has_content = len(response) > 50
        # 메모리 지표 확인
        has_memory_indicators = any(word in response.lower() for word in 
            ['earlier', 'mentioned', 'discussed', 'previously', 'you', 'we', 'our'])
        return "✅ PASS" if (has_content and has_memory_indicators) else "❌ FAIL"
    
    def validate_session_memory(self, response):
        """Agent가 세션 내에서 컨텍스트를 유지하는지 확인"""
        has_memory_content = len(response) > 100 and any(word in response.lower() for word in 
            ['previous', 'earlier', 'mentioned', 'discussed', 'before', 'already'])
        return "✅ PASS" if has_memory_content else "❌ FAIL"
    
    def validate_cross_reference(self, response):
        """Agent가 현재 쿼리를 이전 컨텍스트와 연결할 수 있는지 확인"""
        # 연결 언어 찾기
        connecting_words = ['relate', 'connection', 'previous', 'earlier', 'discussed', 
                           'mentioned', 'context', 'based on', 'as we', 'as i']
        has_connection = any(word in response.lower() for word in connecting_words)
        has_substance = len(response) > 80
        return "✅ PASS" if (has_connection and has_substance) else "❌ FAIL"
    
    def run_validation_summary(self, test_results):
        print("🧪 COMPREHENSIVE TEST VALIDATION SUMMARY")
        print("=" * 60)
        
        total_tests = len(test_results)
        passed_tests = sum(1 for result in test_results.values() if "PASS" in result)
        pass_rate = (passed_tests / total_tests * 100) if total_tests > 0 else 0
        
        for test_name, result in test_results.items():
            print(f"{test_name}: {result}")
        
        print("=" * 60)
        print(f"📊 Overall Pass Rate: {passed_tests}/{total_tests} ({pass_rate:.1f}%)")
        
        if pass_rate >= 80:
            print("✅ EXCELLENT: Memory integration working correctly!")
        elif pass_rate >= 60:
            print("⚠️  GOOD: Most memory features working, some issues to investigate")
        else:
            print("❌ NEEDS ATTENTION: Memory integration has significant issues")
        
        return pass_rate

validator = TestValidator()
print("✅ Validation functions loaded!")

In [ ]:
# 모든 검증 테스트 실행
test_results = {}

# 테스트 1: 메모리 회상 - agent가 논의된 내용을 회상할 수 있는가?
response1 = await agent_y2q1.run("What have we discussed so far in this session?", memory=memory_y2q1)
test_results['Memory Recall'] = validator.validate_memory_recall(str(response1))
print(f"Response 1 length: {len(str(response1))} chars")

# 테스트 2: 세션 메모리 - agent가 컨텍스트를 유지하는가?
response2 = await agent_y2q1.run("What did we talk about earlier?", memory=memory_y2q1)
test_results['Session Memory'] = validator.validate_session_memory(str(response2))
print(f"Response 2 length: {len(str(response2))} chars")

# 테스트 3: 크로스 레퍼런스 기능 - 이전 컨텍스트와 연결할 수 있는가?
response3 = await agent_y2q1.run("How does this relate to what we discussed before?", memory=memory_y2q1)
test_results['Cross Reference'] = validator.validate_cross_reference(str(response3))
print(f"Response 3 length: {len(str(response3))} chars")

# 결과 표시
validator.run_validation_summary(test_results)

## 요약

이 노트북에서는 다음을 시연했습니다:

✅ **장기 메모리 통합**: 크로스 세션 자산 관리를 위해 LlamaIndex와 함께 AgentCore Memory 사용

✅ **투자 여정 추적**: 여러 분기에 걸친 포트폴리오 진화 및 성과 기여도 분석

✅ **시장 인텔리전스**: 시장 인사이트 및 포트폴리오 적용의 시맨틱 검색

✅ **투자 논제 진화**: 초기 포지셔닝에서 시장 적응형 전략으로의 자연스러운 진행

✅ **성과 기여도 분석**: 전술적 결정 및 투자 결과에 대한 상세한 추적

✅ **자산 관리 우수성**: 시간 경과에 따른 포괄적인 고객 관계 및 포트폴리오 최적화

Investment Portfolio Advisor는 장기 메모리가 Advisor를 시간이 지남에 따라 더 똑똑해지는 지속적인 자산 관리 파트너로 변환하여 완전한 투자 히스토리를 유지하고 확장된 고객 관계 전반에 걸쳐 정교한 금융 지식 검색을 가능하게 하는 방법을 보여줍니다.

## 정리

이 노트북에서 사용된 리소스를 정리하기 위해 메모리를 삭제하겠습니다:

**참고**: 메모리를 영구적으로 삭제하려는 경우에만 실행하세요. memory_id 변수는 이 노트북의 앞부분에서 생성된 메모리의 ID를 포함해야 합니다.

In [ ]:
# AgentCore Memory 리소스 정리
try:
    from bedrock_agentcore.memory import MemoryClient
    
    client = MemoryClient(region_name=region)
    client.delete_memory(memory_id)
    print(f"✅ Successfully deleted memory: {memory_id}")
    
except NameError as e:
    print(f"⚠️  Variable not defined: {e}")
    print("Run the notebook from the beginning or set variables manually:")
    print("# memory_id = 'your-memory-id-here'")
    print("# region = 'us-east-1'")
except Exception as e:
    print(f"❌ Error deleting memory: {e}")